In [5]:
import os
import sys

import anndata as nd
import numpy as np
import torch
import pandas as pd

sys.path.append("../")

from data.original import pertdata as pt  # noqa: E402

norman = pt.PertData.from_repo(name="norman", save_dir="original")




##### Outputs -> pandas, numpy, pytorch, csv |||| .pd, .np, .pt, .csv


print("Creating Data Splits")


Dataset directory already exists: original/norman
Loading dataset: norman
Creating Data Splits


In [6]:
obs = norman.adata.obs

In [7]:
X = pd.DataFrame(norman.adata.X.toarray(), columns=norman.adata.var["gene_name"], index=norman.adata.obs.index)
X

gene_name,RP11-34P13.8,RP11-54O7.3,SAMD11,PERM1,HES4,ISG15,RP11-54O7.18,RNF223,LINC01342,TTLL10-AS1,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC136616.1,AC023491.2
cell_barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGGCATGTG-1,0.0,0.0,0.0,0.0,0.000000,0.681502,0.0,0.0,0.0,0.0,...,4.253567,5.419092,3.690722,1.368775,4.551527,1.925863,2.176427,3.868876,0.0,0.0
AAACCTGAGGCCCTTG-1,0.0,0.0,0.0,0.0,0.000000,0.448936,0.0,0.0,0.0,0.0,...,4.328058,5.202683,3.759609,1.602717,4.589678,2.415877,1.897086,3.571369,0.0,0.0
AAACCTGCACGAAGCA-1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,4.380425,5.377383,3.458815,2.162728,4.866675,2.671797,1.072852,3.723229,0.0,0.0
AAACCTGCAGACGTAG-1,0.0,0.0,0.0,0.0,0.892662,0.892662,0.0,0.0,0.0,0.0,...,4.438100,5.616436,4.021444,2.266930,5.045059,3.119025,0.000000,3.789824,0.0,0.0
AAACCTGCAGCCTTGG-1,0.0,0.0,0.0,0.0,0.529184,0.529184,0.0,0.0,0.0,0.0,...,4.122066,5.163007,3.976236,2.160216,4.533418,2.439141,2.376360,3.757771,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTCATGCAT-8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,4.381725,5.337879,4.090175,2.163893,4.663251,2.450516,2.604212,4.090175,0.0,0.0
TTTGTCATCAGTACGT-8,0.0,0.0,0.0,0.0,0.000000,0.638550,0.0,0.0,0.0,0.0,...,3.915124,5.150958,3.628148,1.025133,4.684424,2.098001,1.699034,3.150311,0.0,0.0
TTTGTCATCCACTCCA-8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,4.055220,5.195970,3.332168,2.047660,4.356030,2.346568,1.856266,3.230768,0.0,0.0


In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
conditions = pt.generate_fixed_perturbation_labels(norman.adata.obs["condition"])
conditions_split = conditions.str.split("+")


mlb = MultiLabelBinarizer()
hot_encoded = pd.DataFrame(mlb.fit_transform(conditions_split), columns=pd.Series(mlb.classes_, name="gene_perturbation"), index=conditions.index).drop("ctrl", axis=1)

y = hot_encoded

y

gene_perturbation,AHR,ARID1A,ARRDC3,ATL1,BAK1,BCL2L11,BCORL1,BPGM,C19orf26,C3orf72,...,TMSB4X,TP73,TSC22D1,UBASH3A,UBASH3B,ZBTB1,ZBTB10,ZBTB25,ZC3HAV1,ZNF318
cell_barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGGCATGTG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
AAACCTGAGGCCCTTG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCTGCACGAAGCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCTGCAGACGTAG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCTGCAGCCTTGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTCATGCAT-8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTCATCAGTACGT-8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTCATCCACTCCA-8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
import random

TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1


N = len(X)

random.seed(42)
indices = list(range(N))
random.shuffle(indices)

print(indices)

val_size = int(VALIDATION_SIZE * N)
test_size = int(TEST_SIZE * N)

X_val = X.iloc[indices[:val_size]]
X_test = X.iloc[indices[val_size : val_size + test_size]]
X_train = X.iloc[indices[val_size + test_size :]]

X_val

[49865, 30994, 43889, 7772, 82308, 65729, 67250, 48842, 63456, 4023, 89771, 46689, 86728, 52512, 82192, 1102, 67599, 13056, 22284, 27861, 8138, 25611, 921, 34653, 59035, 74152, 18699, 42779, 25004, 22090, 57666, 49852, 73468, 70491, 9948, 63056, 27722, 13379, 73269, 53298, 8195, 88950, 73833, 76256, 61992, 21861, 45975, 46561, 62806, 1748, 18715, 88326, 34437, 61035, 54295, 36279, 5691, 67210, 14911, 59046, 80942, 88138, 33208, 63357, 52753, 47773, 77447, 78800, 11268, 45821, 27301, 64632, 26116, 47114, 80526, 91032, 62529, 41303, 16392, 3985, 10160, 78277, 316, 3900, 46327, 37077, 59724, 39279, 41266, 33677, 30601, 40018, 27209, 40829, 49645, 39927, 46359, 30332, 20586, 17310, 180, 58644, 57776, 74812, 3449, 61664, 63838, 57990, 70252, 54547, 573, 66028, 37901, 14793, 9368, 40821, 43875, 19236, 46767, 9796, 85615, 9588, 76411, 8197, 19132, 73533, 76848, 13488, 69499, 203, 63796, 43638, 14295, 10565, 74343, 12792, 34696, 2217, 18662, 14118, 6742, 61053, 7793, 76231, 27818, 46978, 69128

gene_name,RP11-34P13.8,RP11-54O7.3,SAMD11,PERM1,HES4,ISG15,RP11-54O7.18,RNF223,LINC01342,TTLL10-AS1,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC136616.1,AC023491.2
cell_barcode,,,,,,,,,,,,,,,,,,,,,
CCTCTGAGTTCGGCAC-5,0.0,0.0,0.0,0.0,0.665054,0.000000,0.0,0.0,0.0,0.0,...,3.471086,4.913223,3.081043,0.000000,4.038323,1.744490,2.146723,3.378880,0.0,0.0
GCCTCTAAGAGAACAG-3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,4.283777,5.198154,3.665196,1.196010,4.497758,1.725227,1.196010,3.946458,0.0,0.0
TACGGGCTCTTCCTTC-4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,4.313108,5.402752,3.720608,1.194606,4.639092,2.526631,1.723572,3.393777,0.0,0.0
GCGCGATTCAACACAC-1,0.0,0.0,0.0,0.0,0.000000,0.683083,0.0,0.0,0.0,0.0,...,3.851395,4.792119,3.461637,1.371159,4.277332,2.122236,1.854710,3.364851,0.0,0.0
CACACCTCAGCGAACA-8,0.0,0.0,0.0,0.0,0.000000,0.800013,0.0,0.0,0.0,0.0,...,4.049629,5.088598,3.647533,1.238704,4.544817,2.584427,2.122670,3.631438,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCTTCCACAGCTGTTA-6,0.0,0.0,0.0,0.0,0.000000,0.763846,0.0,0.0,0.0,0.0,...,3.951627,4.960006,3.464358,1.000549,4.177534,2.064213,1.988673,3.221944,0.0,0.0
CTGATCCGTCCATCCT-3,0.0,0.0,0.0,0.0,0.827923,0.497303,0.0,0.0,0.0,0.0,...,3.949283,5.039149,3.973808,1.274558,4.393099,2.007247,2.480963,3.502972,0.0,0.0
GACCAATAGATGTTAG-2,0.0,0.0,0.0,0.0,0.452467,0.999366,0.0,0.0,0.0,0.0,...,4.044353,5.012903,3.018195,0.762847,4.622074,1.987059,1.610498,3.564766,0.0,0.0
